In [1]:
import argparse, csv
from tensorflow import keras
from tensorflow.python.keras.models import model_from_json
from keras.utils import to_categorical
import pandas as pd

from tools.metadata_tools import *

Using TensorFlow backend.


In [2]:
test = '../Results/json_detection'
train = '../Results/json_detection/train'

matrix_metadata = metadata_to_matrix(test, "json")
test_data = matrix_metadata[:,:14].astype(np.float32)
test_label = to_categorical(matrix_metadata[:,14].astype(np.float32).astype(np.int8))

matrix_metadata = metadata_to_matrix(train, "json")
train_data = matrix_metadata[:,:14].astype(np.float32)
train_label = to_categorical(matrix_metadata[:,14].astype(np.float32).astype(np.int8))

In [3]:
data = np.vstack((train_data,test_data))

In [4]:
label = np.vstack((train_label,test_label))

In [12]:
from monumai.monument import Monument
names = [el for sublist in list(Monument.ELEMENT_DIC.values()) for el in sublist]
indices_style = {}
for style in Monument.TRUE_ELEMENT_DIC:
    indices_style[style] = []

for style, el in Monument.TRUE_ELEMENT_DIC.items():
    for sub_el in el:
        indices_style[style].append(names.index(sub_el))

for style in Monument.TRUE_ELEMENT_DIC:
    indices_style[style] = set(indices_style[style])       

In [13]:
style_names = list(Monument.TRUE_ELEMENT_DIC.keys())

In [14]:
representation_occurences = np.zeros((data.shape[1],label.shape[1]))
representation_values = np.zeros((data.shape[1],label.shape[1]))

In [20]:
indices = set([i for i in range(data.shape[1])])
for i in range(len(data)):
    true_label = np.argmax(label[i])
    style = style_names[true_label]
    true_elements = indices_style[style]
    list_difference = list(indices - true_elements)
    representation_values[list_difference,true_label] += data[i][list_difference]
    representation_occurences[list_difference,true_label] += data[i][list_difference] > 0


In [22]:
representation_values

array([[  0.        ,   7.93312337,  10.82527051,  11.51501596],
       [  0.        ,   5.68943088,   2.37646742,   7.74058412],
       [  0.        ,   6.04744983,  17.04655463,   3.74231929],
       [ 24.46317827,   0.        ,  13.73966838,   8.74879582],
       [  5.70058899,   0.        ,   1.77821945,   2.43604083],
       [  0.92687266,   0.        ,   2.9917349 ,   2.89418342],
       [  5.52238376,  24.25793812,   0.        , 648.55827139],
       [  2.57628921,   3.87499732,   0.        , 287.25203945],
       [  7.60536904,   6.3189718 ,   0.        ,  71.92510768],
       [170.73663795, 130.82209604,   0.        ,   0.        ],
       [ 18.15759289,  50.81673783,   0.        ,   0.        ],
       [  9.22777823,  14.95262958,   0.        ,   0.        ],
       [  4.38107014,   7.02922674,  67.52791598,   0.        ],
       [  2.53558218,   1.6680401 ,  14.96255521,   0.        ]])